# Tutorial 11

## Fitting with SciPy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as constants
from scipy.optimize import curve_fit
from scipy import stats

### Physical constants

1. Using ```scipy.constants```, print the numerical values of:
- speed of light
- Planck constant
- Boltzmann constant
- electron mass

<br>

<br>

<br>

2. What are the units of these quantities?

<br>

<br>

3. Why is it preferable to use ```scipy.constants``` rather than hard-coding numerical values?

<br>

<br>

4. Using ```scipy.constants```, compute:
$E=h\nu$
for visible light with a wavelength of 500 nm. Give the value in electronvolts. ($1~\rm{eV}=1.6\times10^{-19}~\rm{J}$)

<br>

<br>

<br>

<br>

5. Using ```scipy.constants```, estimate the thermal energy $k_BT$ at room temperature (300 K) in electronvolts.

<br>

<br>

<br>

<br>

### Synthetic data

In this part, we want to generate data in order to understand how fitting works.

6. Why is it important to fix a random seed when generating data?

In [ ]:
np.random.seed(42)

<br>

<br>

<br>

<br>

We consider the linear model $y(x)=ax+b$ and generate synthetic data by choosing true values for a and b ($a_{true}=2.5$, $b_{true}=-1.0$). We sample 30 points in x between 0 and 10.

7. How to add Gaussian noise with known standard deviation?

Hint: use ```np.random.normal```

In [ ]:
a_true = 2.5
b_true = -1.0

x = np.linspace(0, 10, 30)
sigma_y = 1.0


<br>

<br>

<br>

<br>

8. Plot the y values.

<br>

<br>

<br>

<br>

### Fitting with ```curve_fit```

This Python function represents the linear model $y(x)=ax+b$:

In [ ]:
def linear_model(x, a, b):
    return a * x + b

9. Use ```scipy.optimize.curve_fit``` to fit the data:
- Without passing uncertainties
- With uncertainties (sigma= argument)

Compare the fitted parameters.

<br>

<br>

<br>

<br>

```curve_fit``` returns two objects:
- the best-fit parameters
- the covariance matrix

10. What does each diagonal element of the covariance matrix represent? Compare for fit with and without uncertainty.

In [ ]:
print("without uncertainties", pcov)
print("With uncertainties", pcov_wsigma)

<br>

<br>

<br>

<br>

11. Compute the 1σ uncertainty on each fitted parameter (obtained with uncertainty) and report the fit results in the form:

$a=a_0\pm\sigma_a$

$b=b_0\pm\sigma_b$

<br>

<br>

<br>

<br>

### Covariance and correlation matrices

From the lecture, we know how to compute the correlation matrix from the covariance matrix:

In [ ]:
def cov2cor(cov):
    '''Convert the covariance matrix to the correlation matrix'''
    D = np.diag(1 / np.sqrt(np.diag(cov)))
    return D @ cov @ D

12. From the covariance matrix of the fit with uncertainty, compute the correlation matrix and display it as a heatmap. How to interpret this heatmap?

<br>

<br>

<br>

<br>

### Fit uncertainty

13. Generate many random parameter sets (a,b) from a multivariate normal distribution using:
- the best-fit parameters as the mean
- the covariance matrix from the fit

<br>

<br>

<br>

<br>

14. Plot several resulting fit curves on top of the data. What do these curves represent physically?

<br>

<br>

<br>

<br>

15. What are we plotting in the following code?

In [ ]:
y_samples = np.array([linear_model(x, a_s, b_s) for a_s, b_s in samples])

y_mean = y_samples.mean(axis=0)
y_std = y_samples.std(axis=0)

plt.errorbar(x, y, yerr=sigma_y, fmt='o')
plt.plot(x, y_mean, color='red')
plt.fill_between(x, y_mean - y_std, y_mean + y_std, color='orange', alpha=0.5)
plt.show()

<br>

<br>

<br>

<br>

### $\chi^2$ and $p$-value

16. Using the best-fit model and the measurement uncertainties, compute the $\chi^2$ value of the fit:

$$\chi^2 = \sum_i \frac{(y_i - y_{model}(x_i))^2}{\sigma_i^2}$$

What does this quantity measure?

<br>

<br>

<br>

<br>

17. Compute the number of degrees of freedom (number of data points - number of model parameters) and the reduced $\chi^2$:

$$\chi_\nu^2=\frac{\chi^2}{DoF}$$

<br>

<br>

<br>

<br>

17. Using ```scipy.stats.chi2```, compute the p-value corresponding to your $\chi^2$.

What does this p-value tell you about the consistency of the model with the data?

<br>

<br>

<br>

<br>

## Exercise

In gamma-ray astronomy, energy spectra are often described by a power-law:

$\frac{dN}{dE}=A\left(\frac{E}{E_0}\right)^{-\gamma}$

We generate a synthetic gamma-ray spectrum and assume a relative uncertainty of 20% on the flux with noise.

In [ ]:
A_true = 1e-11     # normalization
gamma_true = 2.5   # spectral index
E0 = 1.0           # reference energy

E = np.logspace(-1, 1, 20)  # energies in log space
flux = A_true * (E / E0) ** (-gamma_true)

sigma_flux = 0.2 * flux
flux_obs = flux + np.random.normal(0, sigma_flux) # noise added

a) Write a Python function implementing the power-law model to be used with ```curve_fit```.

b) Fit the synthetic spectrum using ```scipy.optimize.curve_fit```, including uncertainties. Inspect the best-fit parameters and the covariance matrix.

We now propagate the parameter uncertainties and correlations from the fit to the model prediction. To do this, we draw multiple parameter sets (A, $\gamma$) from a multivariate normal distribution defined by the best-fit parameters and their covariance matrix.

In [ ]:
samples_g = np.random.multivariate_normal(popt_g, pcov_g, size=300)

c) Evaluate the model for each sampled parameter set and use the ensemble of spectra to estimate the mean model prediction and its 1σ uncertainty as a function of energy.

In [ ]:
E_plot = np.logspace(-1, 1, 100)
flux_samples = np.array([power_law(E_plot, *p) for p in samples_g])

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Energy')
plt.ylabel('Flux')
plt.legend()
plt.show()